In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import time
import math

In [2]:
MON = pd.read_excel('OViN2017_Databestand.xlsx', sheet_name = 'Databestand')

In [3]:
MON1 = MON

In [4]:
number_persons = len(list(MON1['OPID'].unique()))

In [5]:
# MON = MON['OPID', 'Jaardag', 'VertUur', 'AankUur', 'VertMin', 'AankMin', 'BVertUur', 'BVertMin', 'BAankUur', 'BAankMin', 'RVertUur', 'RVertMin', 'RAankUur', 'RAankMin', 'Rvm', 'Doel']    Hier nog alle nuttige kolommen selecteren
MON1 = MON1[MON1['Rvm'] == 6]
MON1 = MON1.reset_index()

In [6]:
MON1['RVertUur'] = MON1['RVertUur'].fillna(MON1['BVertUur'])
MON1['RVertMin'] = MON1['RVertMin'].fillna(MON1['BVertMin'])
MON1['RAankUur'] = MON1['RAankUur'].fillna(MON1['BAankUur'])
MON1['RAankMin'] = MON1['RAankMin'].fillna(MON1['BAankMin'])

MON1['RVertUur'] = MON1['RVertUur'].fillna(MON1['VertUur'])
MON1['RVertMin'] = MON1['RVertMin'].fillna(MON1['VertMin'])
MON1['RAankUur'] = MON1['RAankUur'].fillna(MON1['AankUur'])
MON1['RAankMin'] = MON1['RAankMin'].fillna(MON1['AankMin'])

MON1 = MON1.dropna(subset=['RVertUur'])
MON1 = MON1.dropna(subset=['RAankUur'])
MON1 = MON1.reset_index()
MON1 = MON1.drop(['level_0', 'index'], axis=1)

In [7]:
MON1.loc[MON1['RAankUur'] > 23, 'RAankUur'] = (MON1['RAankUur'] - 24)

In [8]:
daylist = list(MON1['Weekdag'].unique())
daylist.sort()

timelist = []
for a in range(0, 24):
    for b in range(0, 59, 15):
        t = dt.time(hour = a, minute = b)
        timelist.append(t)

personlist = list(MON1['OPID'].unique())

In [10]:
ind = list(range(96))
col = ['Home', 'Work', 'Work Visit', 'Work Transport', 'Bringing', 'Education', 'Shop', 'Visit', 'Wandering', 'Hobby', 'Leisure', 'Personal Care', 'Other', 'Driving']
df9 = pd.DataFrame(index = ind, columns = col)

T = ['00:00', '00:15', '00:30', '00:45', '01:00', '01:15', '01:30', '01:45', '02:00', '02:15', '02:30', '02:45', '03:00', '03:15', '03:30', '03:45', '04:00', '04:15', '04:30', '04:45', '05:00', '05:15', '05:30', '05:45', '06:00', '06:15', '06:30', '06:45', '07:00', '07:15', '07:30', '07:45', '08:00', '08:15', '08:30', '08:45', '09:00', '09:15', '09:30', '09:45', '10:00', '10:15', '10:30', '10:45', '11:00', '11:15', '11:30', '11:45', '12:00', '12:15', '12:30', '12:45', '13:00', '13:15', '13:30', '13:45', '14:00', '14:15', '14:30', '14:45', '15:00', '15:15', '15:30', '15:45', '16:00', '16:15', '16:30', '16:45', '17:00', '17:15', '17:30', '17:45', '18:00', '18:15', '18:30', '18:45', '19:00', '19:15', '19:30', '19:45', '20:00', '20:15', '20:30', '20:45', '21:00', '21:15', '21:30', '21:45', '22:00', '22:15', '22:30', '22:45', '23:00', '23:15', '23:30', '23:45', ]
T7 = T * 7

In [19]:
df10 = pd.DataFrame(index = [], columns = col)

for d in daylist:
    Index_Teller = 0
    print(d)
    df9 = pd.DataFrame(index = ind, columns = col)
    df1 = MON1[MON1['Weekdag'] == d]
    df3 = MON[MON['Weekdag'] == d]
    number_respondents = len(list(df3['OPID'].unique()))
    number_respondents_driving = len(list(df1['OPID'].unique()))
    L3 = [number_respondents] * 96
    L4 = [number_respondents_driving] * 96
    
    personlist = list(df1['OPID'].unique())
    for t in timelist:
        print(t)
        number_home = 0
        number_working = 0
        number_work_visiting = 0
        number_work_transport = 0
        number_bringing = 0
        number_education = 0
        number_shopping = 0
        number_visiting = 0
        number_wandering = 0
        number_hobbying = 0
        number_leisure = 0
        number_personal_care = 0
        number_other = 0
        number_driving = 0
        for p in personlist:
            df2 = df1[df1['OPID'] == p]
            df2 = df2.reset_index()
            if math.isnan(df2.loc[0, 'RVertUur']):
                number_home += 1
            elif t < dt.time(hour = int(df2.loc[0, 'RVertUur']), minute = int(df2.loc[0, 'RVertMin'])):
                number_home += 1
            elif t >= dt.time(hour = int(df2.loc[(df2.shape[0] - 1), 'RAankUur']), minute = int(df2.loc[(df2.shape[0] - 1), 'RAankMin'])):
                number_home += 1                                                      # Als ze altijd thuis slapen?
            elif t < dt.time(hour = int(df2.loc[0, 'RAankUur']), minute = int(df2.loc[0, 'RAankMin'])):
                number_driving += 1
            elif t < dt.time(hour = int(df2.loc[1, 'RVertUur']), minute = int(df2.loc[1, 'RVertMin'])):
                if df2.loc[0, 'Doel'] == 1:
                    number_home += 1
                elif df2.loc[0, 'Doel'] == 2:
                    number_working += 1
                elif df2.loc[0, 'Doel'] == 3:
                    number_work_visiting += 1
                elif df2.loc[0, 'Doel'] == 4:
                    number_work_transport += 1
                elif df2.loc[0, 'Doel'] == 5 or df2.loc[0, 'Doel'] == 6:
                    number_bringing += 1
                elif df2.loc[0, 'Doel'] == 7:
                    number_education += 1
                elif df2.loc[0, 'Doel'] == 8:
                    number_shopping += 1
                elif df2.loc[0, 'Doel'] == 9:
                    number_visiting += 1
                elif df2.loc[0, 'Doel'] == 10:
                    number_wandering += 1
                elif df2.loc[0, 'Doel'] == 11:
                    number_hobbying += 1
                elif df2.loc[0, 'Doel'] == 12:
                    number_leisure += 1
                elif df2.loc[0, 'Doel'] == 13:
                    number_personal_care += 1
                else: number_other += 1
                    
            elif t < dt.time(hour = int(df2.loc[1, 'RAankUur']), minute = int(df2.loc[1, 'RAankMin'])):
                number_driving += 1
            elif t < dt.time(hour = int(df2.loc[2, 'RVertUur']), minute = int(df2.loc[2, 'RVertMin'])):
                if df2.loc[1, 'Doel'] == 1:
                    number_home += 1
                elif df2.loc[1, 'Doel'] == 2:
                    number_working += 1
                elif df2.loc[1, 'Doel'] == 3:
                    number_work_visiting += 1
                elif df2.loc[1, 'Doel'] == 4:
                    number_work_transport += 1
                elif df2.loc[1, 'Doel'] == 5 or df2.loc[0, 'Doel'] == 6:
                    number_bringing += 1
                elif df2.loc[1, 'Doel'] == 7:
                    number_education += 1
                elif df2.loc[1, 'Doel'] == 8:
                    number_shopping += 1
                elif df2.loc[1, 'Doel'] == 9:
                    number_visiting += 1
                elif df2.loc[1, 'Doel'] == 10:
                    number_wandering += 1
                elif df2.loc[1, 'Doel'] == 11:
                    number_hobbying += 1
                elif df2.loc[1, 'Doel'] == 12:
                    number_leisure += 1
                elif df2.loc[1, 'Doel'] == 13:
                    number_personal_care += 1
                else: number_other += 1
               
            elif t < dt.time(hour = int(df2.loc[2, 'RAankUur']), minute = int(df2.loc[2, 'RAankMin'])):
                number_driving += 1
            elif t < dt.time(hour = int(df2.loc[3, 'RVertUur']), minute = int(df2.loc[3, 'RVertMin'])):
                if df2.loc[2, 'Doel'] == 1:
                    number_home += 1
                elif df2.loc[2, 'Doel'] == 2:
                    number_working += 1
                elif df2.loc[2, 'Doel'] == 3:
                    number_work_visiting += 1
                elif df2.loc[2, 'Doel'] == 4:
                    number_work_transport += 1
                elif df2.loc[2, 'Doel'] == 5 or df2.loc[0, 'Doel'] == 6:
                    number_bringing += 1
                elif df2.loc[2, 'Doel'] == 7:
                    number_education += 1
                elif df2.loc[2, 'Doel'] == 8:
                    number_shopping += 1
                elif df2.loc[2, 'Doel'] == 9:
                    number_visiting += 1
                elif df2.loc[2, 'Doel'] == 10:
                    number_wandering += 1
                elif df2.loc[2, 'Doel'] == 11:
                    number_hobbying += 1
                elif df2.loc[2, 'Doel'] == 12:
                    number_leisure += 1
                elif df2.loc[2, 'Doel'] == 13:
                    number_personal_care += 1
                else: number_other += 1
                    
            elif t < dt.time(hour = int(df2.loc[3, 'RAankUur']), minute = int(df2.loc[3, 'RAankMin'])):
                number_driving += 1
            elif t < dt.time(hour = int(df2.loc[4, 'RVertUur']), minute = int(df2.loc[4, 'RVertMin'])):
                if df2.loc[3, 'Doel'] == 1:
                    number_home += 1
                elif df2.loc[3, 'Doel'] == 2:
                    number_working += 1
                elif df2.loc[3, 'Doel'] == 3:
                    number_work_visiting += 1
                elif df2.loc[3, 'Doel'] == 4:
                    number_work_transport += 1
                elif df2.loc[3, 'Doel'] == 5 or df2.loc[0, 'Doel'] == 6:
                    number_bringing += 1
                elif df2.loc[3, 'Doel'] == 7:
                    number_education += 1
                elif df2.loc[3, 'Doel'] == 8:
                    number_shopping += 1
                elif df2.loc[3, 'Doel'] == 9:
                    number_visiting += 1
                elif df2.loc[3, 'Doel'] == 10:
                    number_wandering += 1
                elif df2.loc[3, 'Doel'] == 11:
                    number_hobbying += 1
                elif df2.loc[3, 'Doel'] == 12:
                    number_leisure += 1
                elif df2.loc[3, 'Doel'] == 13:
                    number_personal_care += 1
                else: number_other += 1
                    
            elif t < dt.time(hour = int(df2.loc[4, 'RAankUur']), minute = int(df2.loc[4, 'RAankMin'])):
                number_driving += 1
            elif t < dt.time(hour = int(df2.loc[5, 'RVertUur']), minute = int(df2.loc[5, 'RVertMin'])):
                if df2.loc[4, 'Doel'] == 1:
                    number_home += 1
                elif df2.loc[4, 'Doel'] == 2:
                    number_working += 1
                elif df2.loc[4, 'Doel'] == 3:
                    number_work_visiting += 1
                elif df2.loc[4, 'Doel'] == 4:
                    number_work_transport += 1
                elif df2.loc[4, 'Doel'] == 5 or df2.loc[0, 'Doel'] == 6:
                    number_bringing += 1
                elif df2.loc[4, 'Doel'] == 7:
                    number_education += 1
                elif df2.loc[4, 'Doel'] == 8:
                    number_shopping += 1
                elif df2.loc[4, 'Doel'] == 9:
                    number_visiting += 1
                elif df2.loc[4, 'Doel'] == 10:
                    number_wandering += 1
                elif df2.loc[4, 'Doel'] == 11:
                    number_hobbying += 1
                elif df2.loc[4, 'Doel'] == 12:
                    number_leisure += 1
                elif df2.loc[4, 'Doel'] == 13:
                    number_personal_care += 1
                else: number_other += 1
                    
            elif t < dt.time(hour = int(df2.loc[5, 'RAankUur']), minute = int(df2.loc[5, 'RAankMin'])):
                number_driving += 1
            elif t < dt.time(hour = int(df2.loc[6, 'RVertUur']), minute = int(df2.loc[6, 'RVertMin'])):
                if df2.loc[5, 'Doel'] == 1:
                    number_home += 1
                elif df2.loc[5, 'Doel'] == 2:
                    number_working += 1
                elif df2.loc[5, 'Doel'] == 3:
                    number_work_visiting += 1
                elif df2.loc[5, 'Doel'] == 4:
                    number_work_transport += 1
                elif df2.loc[5, 'Doel'] == 5 or df2.loc[0, 'Doel'] == 6:
                    number_bringing += 1
                elif df2.loc[5, 'Doel'] == 7:
                    number_education += 1
                elif df2.loc[5, 'Doel'] == 8:
                    number_shopping += 1
                elif df2.loc[5, 'Doel'] == 9:
                    number_visiting += 1
                elif df2.loc[5, 'Doel'] == 10:
                    number_wandering += 1
                elif df2.loc[5, 'Doel'] == 11:
                    number_hobbying += 1
                elif df2.loc[5, 'Doel'] == 12:
                    number_leisure += 1
                elif df2.loc[5, 'Doel'] == 13:
                    number_personal_care += 1
                else: number_other += 1
                    
            elif t < dt.time(hour = int(df2.loc[6, 'RAankUur']), minute = int(df2.loc[6, 'RAankMin'])):
                number_driving += 1
            elif t < dt.time(hour = int(df2.loc[7, 'RVertUur']), minute = int(df2.loc[7, 'RVertMin'])):
                if df2.loc[6, 'Doel'] == 1:
                    number_home += 1
                elif df2.loc[6, 'Doel'] == 2:
                    number_working += 1
                elif df2.loc[6, 'Doel'] == 3:
                    number_work_visiting += 1
                elif df2.loc[6, 'Doel'] == 4:
                    number_work_transport += 1
                elif df2.loc[6, 'Doel'] == 5 or df2.loc[0, 'Doel'] == 6:
                    number_bringing += 1
                elif df2.loc[6, 'Doel'] == 7:
                    number_education += 1
                elif df2.loc[6, 'Doel'] == 8:
                    number_shopping += 1
                elif df2.loc[6, 'Doel'] == 9:
                    number_visiting += 1
                elif df2.loc[6, 'Doel'] == 10:
                    number_wandering += 1
                elif df2.loc[6, 'Doel'] == 11:
                    number_hobbying += 1
                elif df2.loc[6, 'Doel'] == 12:
                    number_leisure += 1
                elif df2.loc[6, 'Doel'] == 13:
                    number_personal_care += 1
                else: number_other += 1
                    
            elif t < dt.time(hour = int(df2.loc[7, 'RAankUur']), minute = int(df2.loc[7, 'RAankMin'])):
                number_driving += 1
            elif t < dt.time(hour = int(df2.loc[8, 'RVertUur']), minute = int(df2.loc[8, 'RVertMin'])):
                if df2.loc[7, 'Doel'] == 1:
                    number_home += 1
                elif df2.loc[7, 'Doel'] == 2:
                    number_working += 1
                elif df2.loc[7, 'Doel'] == 3:
                    number_work_visiting += 1
                elif df2.loc[7, 'Doel'] == 4:
                    number_work_transport += 1
                elif df2.loc[7, 'Doel'] == 5 or df2.loc[0, 'Doel'] == 6:
                    number_bringing += 1
                elif df2.loc[7, 'Doel'] == 7:
                    number_education += 1
                elif df2.loc[7, 'Doel'] == 8:
                    number_shopping += 1
                elif df2.loc[7, 'Doel'] == 9:
                    number_visiting += 1
                elif df2.loc[7, 'Doel'] == 10:
                    number_wandering += 1
                elif df2.loc[7, 'Doel'] == 11:
                    number_hobbying += 1
                elif df2.loc[7, 'Doel'] == 12:
                    number_leisure += 1
                elif df2.loc[7, 'Doel'] == 13:
                    number_personal_care += 1
                else: number_other += 1
                    
            elif t < dt.time(hour = int(df2.loc[8, 'RAankUur']), minute = int(df2.loc[8, 'RAankMin'])):
                number_driving += 1
            elif t < dt.time(hour = int(df2.loc[9, 'RVertUur']), minute = int(df2.loc[9, 'RVertMin'])):
                if df2.loc[8, 'Doel'] == 1:
                    number_home += 1
                elif df2.loc[8, 'Doel'] == 2:
                    number_working += 1
                elif df2.loc[8, 'Doel'] == 3:
                    number_work_visiting += 1
                elif df2.loc[8, 'Doel'] == 4:
                    number_work_transport += 1
                elif df2.loc[8, 'Doel'] == 5 or df2.loc[0, 'Doel'] == 6:
                    number_bringing += 1
                elif df2.loc[8, 'Doel'] == 7:
                    number_education += 1
                elif df2.loc[8, 'Doel'] == 8:
                    number_shopping += 1
                elif df2.loc[8, 'Doel'] == 9:
                    number_visiting += 1
                elif df2.loc[8, 'Doel'] == 10:
                    number_wandering += 1
                elif df2.loc[8, 'Doel'] == 11:
                    number_hobbying += 1
                elif df2.loc[8, 'Doel'] == 12:
                    number_leisure += 1
                elif df2.loc[8, 'Doel'] == 13:
                    number_personal_care += 1
                else: number_other += 1
                    
            elif t < dt.time(hour = int(df2.loc[9, 'RAankUur']), minute = int(df2.loc[9, 'RAankMin'])):
                number_driving += 1
            elif t < dt.time(hour = int(df2.loc[10, 'RVertUur']), minute = int(df2.loc[10, 'RVertMin'])):
                if df2.loc[9, 'Doel'] == 1:
                    number_home += 1
                elif df2.loc[9, 'Doel'] == 2:
                    number_working += 1
                elif df2.loc[9, 'Doel'] == 3:
                    number_work_visiting += 1
                elif df2.loc[9, 'Doel'] == 4:
                    number_work_transport += 1
                elif df2.loc[9, 'Doel'] == 5 or df2.loc[0, 'Doel'] == 6:
                    number_bringing += 1
                elif df2.loc[9, 'Doel'] == 7:
                    number_education += 1
                elif df2.loc[9, 'Doel'] == 8:
                    number_shopping += 1
                elif df2.loc[9, 'Doel'] == 9:
                    number_visiting += 1
                elif df2.loc[9, 'Doel'] == 10:
                    number_wandering += 1
                elif df2.loc[9, 'Doel'] == 11:
                    number_hobbying += 1
                elif df2.loc[9, 'Doel'] == 12:
                    number_leisure += 1
                elif df2.loc[9, 'Doel'] == 13:
                    number_personal_care += 1
                else: number_other += 1
                    
            elif t < dt.time(hour = int(df2.loc[10, 'RAankUur']), minute = int(df2.loc[10, 'RAankMin'])):
                number_driving += 1
            elif t < dt.time(hour = int(df2.loc[11, 'RVertUur']), minute = int(df2.loc[11, 'RVertMin'])):
                if df2.loc[10, 'Doel'] == 1:
                    number_home += 1
                elif df2.loc[10, 'Doel'] == 2:
                    number_working += 1
                elif df2.loc[10, 'Doel'] == 3:
                    number_work_visiting += 1
                elif df2.loc[10, 'Doel'] == 4:
                    number_work_transport += 1
                elif df2.loc[10, 'Doel'] == 5 or df2.loc[0, 'Doel'] == 6:
                    number_bringing += 1
                elif df2.loc[10, 'Doel'] == 7:
                    number_education += 1
                elif df2.loc[10, 'Doel'] == 8:
                    number_shopping += 1
                elif df2.loc[10, 'Doel'] == 9:
                    number_visiting += 1
                elif df2.loc[10, 'Doel'] == 10:
                    number_wandering += 1
                elif df2.loc[10, 'Doel'] == 11:
                    number_hobbying += 1
                elif df2.loc[10, 'Doel'] == 12:
                    number_leisure += 1
                elif df2.loc[10, 'Doel'] == 13:
                    number_personal_care += 1
                else: number_other += 1
                    
            elif t < dt.time(hour = int(df2.loc[11, 'RAankUur']), minute = int(df2.loc[11, 'RAankMin'])):
                number_driving += 1
            elif t < dt.time(hour = int(df2.loc[12, 'RVertUur']), minute = int(df2.loc[12, 'RVertMin'])):
                if df2.loc[11, 'Doel'] == 1:
                    number_home += 1
                elif df2.loc[11, 'Doel'] == 2:
                    number_working += 1
                elif df2.loc[11, 'Doel'] == 3:
                    number_work_visiting += 1
                elif df2.loc[11, 'Doel'] == 4:
                    number_work_transport += 1
                elif df2.loc[11, 'Doel'] == 5 or df2.loc[0, 'Doel'] == 6:
                    number_bringing += 1
                elif df2.loc[11, 'Doel'] == 7:
                    number_education += 1
                elif df2.loc[11, 'Doel'] == 8:
                    number_shopping += 1
                elif df2.loc[11, 'Doel'] == 9:
                    number_visiting += 1
                elif df2.loc[11, 'Doel'] == 10:
                    number_wandering += 1
                elif df2.loc[11, 'Doel'] == 11:
                    number_hobbying += 1
                elif df2.loc[11, 'Doel'] == 12:
                    number_leisure += 1
                elif df2.loc[11, 'Doel'] == 13:
                    number_personal_care += 1
                else: number_other += 1
                    
            elif t < dt.time(hour = int(df2.loc[12, 'RAankUur']), minute = int(df2.loc[12, 'RAankMin'])):
                number_driving += 1
            elif t < dt.time(hour = int(df2.loc[13, 'RVertUur']), minute = int(df2.loc[13, 'RVertMin'])):
                if df2.loc[12, 'Doel'] == 1:
                    number_home += 1
                elif df2.loc[12, 'Doel'] == 2:
                    number_working += 1
                elif df2.loc[12, 'Doel'] == 3:
                    number_work_visiting += 1
                elif df2.loc[12, 'Doel'] == 4:
                    number_work_transport += 1
                elif df2.loc[12, 'Doel'] == 5 or df2.loc[0, 'Doel'] == 6:
                    number_bringing += 1
                elif df2.loc[12, 'Doel'] == 7:
                    number_education += 1
                elif df2.loc[12, 'Doel'] == 8:
                    number_shopping += 1
                elif df2.loc[12, 'Doel'] == 9:
                    number_visiting += 1
                elif df2.loc[12, 'Doel'] == 10:
                    number_wandering += 1
                elif df2.loc[12, 'Doel'] == 11:
                    number_hobbying += 1
                elif df2.loc[12, 'Doel'] == 12:
                    number_leisure += 1
                elif df2.loc[12, 'Doel'] == 13:
                    number_personal_care += 1
                else: number_other += 1
                    
            elif t < dt.time(hour = int(df2.loc[13, 'RAankUur']), minute = int(df2.loc[13, 'RAankMin'])):
                number_driving += 1
            elif t < dt.time(hour = int(df2.loc[14, 'RVertUur']), minute = int(df2.loc[14, 'RVertMin'])):
                if df2.loc[13, 'Doel'] == 1:
                    number_home += 1
                elif df2.loc[13, 'Doel'] == 2:
                    number_working += 1
                elif df2.loc[13, 'Doel'] == 3:
                    number_work_visiting += 1
                elif df2.loc[13, 'Doel'] == 4:
                    number_work_transport += 1
                elif df2.loc[13, 'Doel'] == 5 or df2.loc[0, 'Doel'] == 6:
                    number_bringing += 1
                elif df2.loc[13, 'Doel'] == 7:
                    number_education += 1
                elif df2.loc[13, 'Doel'] == 8:
                    number_shopping += 1
                elif df2.loc[13, 'Doel'] == 9:
                    number_visiting += 1
                elif df2.loc[13, 'Doel'] == 10:
                    number_wandering += 1
                elif df2.loc[13, 'Doel'] == 11:
                    number_hobbying += 1
                elif df2.loc[13, 'Doel'] == 12:
                    number_leisure += 1
                elif df2.loc[13, 'Doel'] == 13:
                    number_personal_care += 1
                else: number_other += 1
                    
            elif t < dt.time(hour = int(df2.loc[14, 'RAankUur']), minute = int(df2.loc[14, 'RAankMin'])):
                number_driving += 1
            elif t < dt.time(hour = int(df2.loc[15, 'RVertUur']), minute = int(df2.loc[15, 'RVertMin'])):
                if df2.loc[14, 'Doel'] == 1:
                    number_home += 1
                elif df2.loc[14, 'Doel'] == 2:
                    number_working += 1
                elif df2.loc[14, 'Doel'] == 3:
                    number_work_visiting += 1
                elif df2.loc[14, 'Doel'] == 4:
                    number_work_transport += 1
                elif df2.loc[14, 'Doel'] == 5 or df2.loc[0, 'Doel'] == 6:
                    number_bringing += 1
                elif df2.loc[14, 'Doel'] == 7:
                    number_education += 1
                elif df2.loc[14, 'Doel'] == 8:
                    number_shopping += 1
                elif df2.loc[14, 'Doel'] == 9:
                    number_visiting += 1
                elif df2.loc[14, 'Doel'] == 10:
                    number_wandering += 1
                elif df2.loc[14, 'Doel'] == 11:
                    number_hobbying += 1
                elif df2.loc[14, 'Doel'] == 12:
                    number_leisure += 1
                elif df2.loc[14, 'Doel'] == 13:
                    number_personal_care += 1
                else: number_other += 1
                    
            elif t < dt.time(hour = int(df2.loc[15, 'RAankUur']), minute = int(df2.loc[15, 'RAankMin'])):
                number_driving += 1
            else: 
                number_home += 1
                print('Something is going wrong')
                
        df9.loc[Index_Teller, 'Home'] = number_home
        df9.loc[Index_Teller, 'Work'] = number_working
        df9.loc[Index_Teller, 'Work Visit'] = number_work_visiting
        df9.loc[Index_Teller, 'Work Transport'] = number_work_transport
        df9.loc[Index_Teller, 'Bringing'] = number_bringing
        df9.loc[Index_Teller, 'Education'] = number_education
        df9.loc[Index_Teller, 'Shop'] = number_shopping
        df9.loc[Index_Teller, 'Visit'] = number_visiting
        df9.loc[Index_Teller, 'Wandering'] = number_wandering
        df9.loc[Index_Teller, 'Hobby'] = number_hobbying
        df9.loc[Index_Teller, 'Leisure'] = number_leisure
        df9.loc[Index_Teller, 'Personal Care'] = number_personal_care
        df9.loc[Index_Teller, 'Other'] = number_other
        df9.loc[Index_Teller, 'Driving'] = number_driving
        
        Index_Teller += 1
    
    df9['Number respondents'] = L3
    df9['Number Respondents Driving'] = L4
        
    df9['index'] = T
    df9 = df9.set_index('index')
    df9.to_csv('Behaviour all cars weekdag dag ' + str(d) + '.csv')
    df10 = df10.append(df9, ignore_index=True)
    
    # Percentages
    df_percentage_day = df9
    for c in col:
        df_percentage_day[c] = df_percentage_day[c] / df_percentage_day['Number Respondents Driving']
    df_percentage_day.to_csv('Percentages weekdag day ' + str(d) + '.csv')
    
df10 = df10.reset_index()
df10['index'] = T7
df10 = df10.set_index('index')
df10.to_csv('Behaviour all cars weekdag alle dagen.csv')

df10_percentages = df10

for c in col:
    df10_percentages[c] = df10_percentages[c] / df10_percentages['Number Respondents Driving']

df10_percentages.to_csv('Percentages weekdag alle dagen.csv')

1
00:00:00
00:15:00
00:30:00
00:45:00
01:00:00
01:15:00
01:30:00
01:45:00
02:00:00
02:15:00
02:30:00
02:45:00
03:00:00
03:15:00
03:30:00
03:45:00
04:00:00
04:15:00
04:30:00
04:45:00
05:00:00
05:15:00
05:30:00
05:45:00
06:00:00
06:15:00
06:30:00
06:45:00
07:00:00
07:15:00
07:30:00
07:45:00
08:00:00
08:15:00
08:30:00
08:45:00
09:00:00
09:15:00
09:30:00
09:45:00
10:00:00
10:15:00
10:30:00
10:45:00
11:00:00
11:15:00
11:30:00
11:45:00
12:00:00
12:15:00
12:30:00
12:45:00
13:00:00
13:15:00
13:30:00
13:45:00
14:00:00
14:15:00
14:30:00
14:45:00
15:00:00
15:15:00
15:30:00
15:45:00
16:00:00
16:15:00
16:30:00
16:45:00
17:00:00
17:15:00
17:30:00
17:45:00
18:00:00
18:15:00
18:30:00
18:45:00
19:00:00
19:15:00
19:30:00
19:45:00
20:00:00
20:15:00
20:30:00
20:45:00
21:00:00
21:15:00
21:30:00
21:45:00
22:00:00
22:15:00
22:30:00
22:45:00
23:00:00
23:15:00
23:30:00
23:45:00
2
00:00:00
00:15:00
00:30:00
00:45:00
01:00:00
01:15:00
01:30:00
01:45:00
02:00:00
02:15:00
02:30:00
02:45:00
03:00:00
03:15:00
03:30:

# Percentages

In [2]:
# df10 = pd.read_csv('Behaviour all cars weekdag alle dagen.csv')

In [5]:
# df10 = df10.rename(columns={'index': 'Time'})

In [47]:
# df_percentages = df10
# df_percentages['Number Respondents'] = df_percentages.sum(axis = 1, skipna = True)

# for i in list(df_percentages.columns):
#     df_percentages[i] = df_percentages[i] / df_percentages['Number Respondents']

In [ ]:
# df_percentages.to_csv('Percentages weekdag alle dagen.csv')

In [ ]:
# number_respondents_list = []
# for d in daylist:
#     df3 = MON[MON['Weekdag'] == d]
#     number_respondents = len(list(df3['OPID'].unique()))
#     L = [number_respondents] * 96
#     number_respondents_list = number_respondents_list + L